In [1]:
import json
import pandas as pd


In [4]:
with open('../dataset/hsreply_data/sorted/decks.json', 'r') as file:
    decks = json.load(file)

# 创建一个空的DataFrame
columns = ['Card Ind', 'Card Count']
df = pd.DataFrame(columns=columns)

# 遍历套牌信息并添加到DataFrame中
for deck in decks:
    deck_id = deck['deck_id']
    deck_list = deck['deck_list']

    for card_info, card_count in deck_list:
        card_ind = card_info

        # 将数据添加到DataFrame中
        df = df.append({
            'Deck ID': deck_id,
            'Card Ind': card_ind,
            'Card Count': card_count
        }, ignore_index=True)

C:\Users\djhxx\AppData\Local\Temp\ipykernel_32328\2529717738.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\djhxx\AppData\Local\Temp\ipykernel_32328\2529717738.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\djhxx\AppData\Local\Temp\ipykernel_32328\2529717738.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\djhxx\AppData\Local\Temp\ipykernel_32328\2529717738.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\djhxx\AppData\Local\Temp\ipykernel_32328\2529717738.py:17: FutureWarning: The frame.append method is deprecated

KeyboardInterrupt: 